In [ ]:
import graph_generator as gg
import obm_dp as dp
import numpy as np

%load_ext autoreload
%autoreload 2

In [ ]:
config = {
    'graph_type': 'GEOM',
    'threshold': 0.0,
    'scaling': 1/ np.sqrt(2)
}

m = 5
n = 5
A = gg.sample_bipartite_graph(m, n, **config)
p = [1 for _ in range(m)]

In [ ]:
cache = dp.cache_stochastic_opt(A, p)
coin_flips = [np.random.binomial(1, _p) for _p in p]

In [ ]:
dp.stochastic_opt(A, coin_flips, cache)

In [ ]:
dp.greedy(A, coin_flips)